### Preparação

In [ ]:
install.packages('gvlma')
install.packages('Hmisc')
install.packages('caret')

library(caret)  # manipulação de dados
library(dplyr)  # analises estatísticas e manipulação
library(gvlma)  # Modelo de regressão logística
library(tidyr)  # manipulação dos dados
library(glue)   # Print especial
library(Hmisc)  

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: ggplot2

Loading required package: lattice

Warning message in system("timedatectl", intern = TRUE):
“running command 'timedatectl' had status 1”

Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘glue’


The following object is masked from ‘package:dplyr’:

    collapse


Loading required package: survival


Attaching package: ‘survival’


The following object is masked from ‘package:caret’:

    cluster


Loading required package: Formula


Attaching package: ‘Hmisc’


The following objects are masked from ‘package:dplyr’:

    src, summarize


T

In [ ]:
dados <- read.csv('/content/titanic_dw_train.csv', sep = ',')
head(dados)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
,<int>,<int>,<int>,<chr>,<chr>,<dbl>,<int>,<int>,<chr>,<dbl>,<chr>,<chr>
1,2,0,3,"Johansson, Mr. Nils",male,29,0,0,347467,7.8542,NA,S
2,3,1,1,"Minahan, Mrs. William Edward (Lillian E Thorpe)",female,37,1,0,19928,90.0000,C78,Q
3,4,0,2,"Hocking, Mr. Samuel James Metcalfe",male,36,0,0,242963,13.0000,NA,S
4,5,1,1,"Chibnall, Mrs. (Edith Martha Bowerman)",female,NA,0,1,113505,55.0000,E33,S
5,6,0,3,"Corn, Mr. Harry",male,30,0,0,SOTON/OQ 392090,8.0500,NA,S
6,10,0,3,"Wiklund, Mr. Karl Johan",male,21,1,0,3101266,6.4958,NA,S


In [ ]:
str(dados)

'data.frame':	891 obs. of  12 variables:
 $ PassengerId: int  2 3 4 5 6 10 11 12 13 15 ...
 $ Survived   : int  0 1 0 1 0 0 1 0 0 1 ...
 $ Pclass     : int  3 1 2 1 3 3 1 3 3 2 ...
 $ Name       : chr  "Johansson, Mr. Nils" "Minahan, Mrs. William Edward (Lillian E Thorpe)" "Hocking, Mr. Samuel James Metcalfe" "Chibnall, Mrs. (Edith Martha Bowerman)" ...
 $ Sex        : chr  "male" "female" "male" "female" ...
 $ Age        : num  29 37 36 NA 30 21 53 20 26 62 ...
 $ SibSp      : int  0 1 0 0 0 1 1 0 0 0 ...
 $ Parch      : int  0 0 0 1 0 0 1 0 0 0 ...
 $ Ticket     : chr  "347467" "19928" "242963" "113505" ...
 $ Fare       : num  7.85 90 13 55 8.05 ...
 $ Cabin      : chr  NA "C78" NA "E33" ...
 $ Embarked   : chr  "S" "Q" "S" "S" ...


In [ ]:
set.seed(1234)
df_divisao <- createDataPartition(dados$Survived, p=0.7, list = F)

In [ ]:
titanic <- dados[df_divisao, ] # 70% das observações
titanic_teste <- dados[-df_divisao, ] # 30% das observações

## a)  Preparando e analisando os dados

Variáveis escolhidas: Survived, Pclass, Sex, Age, SibSp, Parch, Fare, Embarked

### i) Selecionando um subconjunto apenas com as variáveis interessantes para a análise

#### Tratamentos dos dados de Treino

In [ ]:
str(titanic)

'data.frame':	624 obs. of  12 variables:
 $ PassengerId: int  2 3 4 5 15 17 18 19 21 22 ...
 $ Survived   : int  0 1 0 1 1 1 1 0 1 1 ...
 $ Pclass     : int  3 1 2 1 2 3 1 2 3 3 ...
 $ Name       : chr  "Johansson, Mr. Nils" "Minahan, Mrs. William Edward (Lillian E Thorpe)" "Hocking, Mr. Samuel James Metcalfe" "Chibnall, Mrs. (Edith Martha Bowerman)" ...
 $ Sex        : chr  "male" "female" "male" "female" ...
 $ Age        : num  29 37 36 NA 62 24 38 27 NA 1 ...
 $ SibSp      : int  0 1 0 0 0 0 0 1 0 1 ...
 $ Parch      : int  0 0 0 1 0 0 0 0 0 2 ...
 $ Ticket     : chr  "347467" "19928" "242963" "113505" ...
 $ Fare       : num  7.85 90 13 55 10.5 ...
 $ Cabin      : chr  NA "C78" NA "E33" ...
 $ Embarked   : chr  "S" "Q" "S" "S" ...


In [ ]:
describe(titanic)

titanic 

 12  Variables      624  Observations
--------------------------------------------------------------------------------
PassengerId 
       n  missing distinct     Info     Mean      Gmd      .05      .10 
     624        0      624        1    660.1    430.8    77.45   137.30 
     .25      .50      .75      .90      .95 
  344.50   668.00   979.75  1171.40  1236.85 

lowest :    2    3    4    5   15, highest: 1294 1296 1298 1303 1304
--------------------------------------------------------------------------------
Survived 
       n  missing distinct     Info      Sum     Mean      Gmd 
     624        0        2    0.712      242   0.3878   0.4756 

--------------------------------------------------------------------------------
Pclass 
       n  missing distinct     Info     Mean      Gmd 
     624        0        3    0.821    2.271   0.8866 
                            
Value          1     2     3
Frequency    164   127   333
Proportion 0.263 0.204 0.534
---------------

In [ ]:
summary(titanic$Age)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
 0.1667 21.0000 28.0000 30.3042 39.0000 80.0000     118 

In [ ]:
colnames(titanic)

[1] "PassengerId" "Survived"    "Pclass"      "Name"        "Sex"        
 [6] "Age"         "SibSp"       "Parch"       "Ticket"      "Fare"       
[11] "Cabin"       "Embarked"

In [ ]:
# Survived, Pclass, Sex, Age, SibSp, Parch, Fare, Embarked
titanic['Name'] <- NULL
titanic['Ticket'] <- NULL
titanic['Cabin'] <- NULL

In [ ]:
head(titanic,5)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
,<int>,<int>,<int>,<chr>,<dbl>,<int>,<int>,<dbl>,<chr>
1,2,0,3,male,29,0,0,7.8542,S
2,3,1,1,female,37,1,0,90.0000,Q
3,4,0,2,male,36,0,0,13.0000,S
4,5,1,1,female,NA,0,1,55.0000,S
10,15,1,2,male,62,0,0,10.5000,S


In [ ]:
tail(titanic,5)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
,<int>,<int>,<int>,<chr>,<dbl>,<int>,<int>,<dbl>,<chr>
884,1294,0,3,female,18,0,1,14.4542,C
886,1296,0,3,male,NA,0,0,7.7750,S
888,1298,0,2,male,22,2,0,31.5000,S
890,1303,0,3,male,31,0,0,7.7750,S
891,1304,0,3,male,21,0,0,8.0500,S


In [ ]:
titanic$Survived <- as.factor(titanic$Survived)

In [ ]:
str(titanic)

'data.frame':	624 obs. of  9 variables:
 $ PassengerId: int  2 3 4 5 15 17 18 19 21 22 ...
 $ Survived   : Factor w/ 2 levels "0","1": 1 2 1 2 2 2 2 1 2 2 ...
 $ Pclass     : int  3 1 2 1 2 3 1 2 3 3 ...
 $ Sex        : chr  "male" "female" "male" "female" ...
 $ Age        : num  29 37 36 NA 62 24 38 27 NA 1 ...
 $ SibSp      : int  0 1 0 0 0 0 0 1 0 1 ...
 $ Parch      : int  0 0 0 1 0 0 0 0 0 2 ...
 $ Fare       : num  7.85 90 13 55 10.5 ...
 $ Embarked   : chr  "S" "Q" "S" "S" ...


In [ ]:
cbind(summary(titanic$Survived))

0,382
1,242


Percebemos, por exemplo, que a nossa amostra da base de treino é composta por 35% de mulheres e 65% de homens, a idade mediana é de 28 anos e 75% tinham menos de 40 anos. Das pessoas que estavam no navio, 26% estavam na 1ª classe, 20% na 2ª e 53% na 3ª, a maioria (415) não tinha irmãos no navio e o restante tinha ao menos 1. A base de treino possui 623 registros e 12 variáveis, destes registros 242 representam os sobreviventes e 381 mortes.

#### Tratamento dos dados de Teste

In [ ]:
str(titanic_teste)

'data.frame':	267 obs. of  12 variables:
 $ PassengerId: int  6 10 11 12 13 24 26 28 33 35 ...
 $ Survived   : int  0 0 1 0 0 0 1 1 1 1 ...
 $ Pclass     : int  3 3 1 3 3 2 1 2 1 1 ...
 $ Name       : chr  "Corn, Mr. Harry" "Wiklund, Mr. Karl Johan" "Dodge, Dr. Washington" "Alhomaki, Mr. Ilmari Rudolf" ...
 $ Sex        : chr  "male" "male" "male" "male" ...
 $ Age        : num  30 21 53 20 26 24 29 14 NA 32 ...
 $ SibSp      : int  0 1 1 0 0 2 0 1 0 0 ...
 $ Parch      : int  0 0 1 0 0 0 0 0 0 0 ...
 $ Ticket     : chr  "SOTON/OQ 392090" "3101266" "33638" "SOTON/O2 3101287" ...
 $ Fare       : num  8.05 6.5 81.86 7.92 7.9 ...
 $ Cabin      : chr  NA NA "A34" NA ...
 $ Embarked   : chr  "S" "S" "S" "S" ...


In [ ]:
describe(titanic_teste)

titanic_teste 

 12  Variables      267  Observations
--------------------------------------------------------------------------------
PassengerId 
       n  missing distinct     Info     Mean      Gmd      .05      .10 
     267        0      267        1    646.8    444.4     58.7    109.2 
     .25      .50      .75      .90      .95 
   320.0    666.0    976.0   1169.2   1251.4 

lowest :    6   10   11   12   13, highest: 1287 1293 1295 1297 1299
--------------------------------------------------------------------------------
Survived 
       n  missing distinct     Info      Sum     Mean      Gmd 
     267        0        2    0.684       94   0.3521   0.4579 

--------------------------------------------------------------------------------
Pclass 
       n  missing distinct     Info     Mean      Gmd 
     267        0        3    0.807    2.345   0.8287 
                            
Value          1     2     3
Frequency     56    63   148
Proportion 0.210 0.236 0.554
---------

In [ ]:
summary(titanic_teste$Age)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
   0.75   19.00   28.00   29.26   39.00   71.00      54 

In [ ]:
# Survived, Pclass, Sex, Age, SibSp, Parch, Fare, Embarked
titanic_teste['Name'] <- NULL
titanic_teste['Ticket'] <- NULL
titanic_teste['Cabin'] <- NULL

In [ ]:
head(titanic_teste,5)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
,<int>,<int>,<int>,<chr>,<dbl>,<int>,<int>,<dbl>,<chr>
5,6,0,3,male,30,0,0,8.0500,S
6,10,0,3,male,21,1,0,6.4958,S
7,11,1,1,male,53,1,1,81.8583,S
8,12,0,3,male,20,0,0,7.9250,S
9,13,0,3,male,26,0,0,7.8958,S


In [ ]:
tail(titanic_teste,5)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
,<int>,<int>,<int>,<chr>,<dbl>,<int>,<int>,<dbl>,<chr>
879,1287,1,1,female,44,0,0,27.7208,C
883,1293,0,1,male,50,1,0,106.4250,C
885,1295,0,3,male,NA,0,0,7.7500,Q
887,1297,1,1,male,36,0,0,26.3875,S
889,1299,1,2,female,24,0,0,13.0000,S


In [ ]:
titanic_teste$Survived <- as.factor(titanic_teste$Survived)

In [ ]:
cbind(summary(titanic_teste$Survived))

0,173
1,94


###  ii) Atribuindo um valor para os NAs na variável Age. 

Neste caso o critério escolhido foi o de aplicar a média entre os membros de mesmo sexo e classe

#### Tratamento Treino

In [ ]:
class_sex <- group_by(titanic, Pclass, Sex) 

class_sex %>% 
summarise(média= round(mean(Age, na.rm = TRUE)), mediana = median(Age, na.rm= TRUE))

`summarise()` has grouped output by 'Pclass'. You can override using the `.groups` argument.



Pclass,Sex,média,mediana
<int>,<chr>,<dbl>,<dbl>
1,female,35,35
1,male,41,42
2,female,28,28
2,male,32,29
3,female,22,22
3,male,26,26


In [ ]:
# Criando a tabela com as medianas por grupo:
AgePclass_medianas <- titanic %>% group_by(Sex,Pclass) %>% summarise(idade = median(Age, na.rm = T),) %>% arrange(Pclass) %>% ungroup()
AgePclass_medianas <- AgePclass_medianas %>% rename(Age=idade) %>% select(Pclass, Sex, Age)

# filtrando somente as linhas com NA em Age
trainMissingAge <- filter(titanic, is.na(Age)) %>% select(PassengerId, Pclass, Sex, Age)

# Join pra ficar com tudo em um data.frame
PassAge <- left_join(trainMissingAge, AgePclass_medianas, by = c("Sex","Pclass")) %>% select(PassengerId, Age=Age.y)

# Join na base train.
train.rawAge <- left_join(titanic, PassAge, by = c("PassengerId"))

# Preenchendo os NAs com mutate
titanic <- mutate(train.rawAge, Age = if_else(is.na(Age.x), Age.y, Age.x)) %>% select(-Age.x, -Age.y)
head(titanic)

`summarise()` has grouped output by 'Sex'. You can override using the `.groups` argument.



,PassengerId,Survived,Pclass,Sex,SibSp,Parch,Fare,Embarked,Age
,<int>,<fct>,<int>,<chr>,<int>,<int>,<dbl>,<chr>,<dbl>
1,2,0,3,male,0,0,7.8542,S,29
2,3,1,1,female,1,0,90.0000,Q,37
3,4,0,2,male,0,0,13.0000,S,36
4,5,1,1,female,0,1,55.0000,S,35
5,15,1,2,male,0,0,10.5000,S,62
6,17,1,3,male,0,0,7.5500,S,24


In [ ]:
sum(is.na(titanic$Age))

[1] 0

#### Tratamento Teste

In [ ]:
class_sex <- group_by(titanic_teste, Pclass, Sex) 

class_sex %>% 
summarise(média= round(mean(Age, na.rm = TRUE)), mediana = median(Age, na.rm= TRUE))

`summarise()` has grouped output by 'Pclass'. You can override using the `.groups` argument.



Pclass,Sex,média,mediana
<int>,<chr>,<dbl>,<dbl>
1,female,39,38.5
1,male,43,48.0
2,female,25,24.0
2,male,30,29.5
3,female,21,22.0
3,male,27,25.5


In [ ]:
# Substituindo os missing values pela mediana das idades por grupo
AgePclass_medianas <- titanic_teste %>% group_by(Sex,Pclass) %>% summarise(idade = median(Age, na.rm = T),) %>% arrange(Pclass) %>% ungroup()
AgePclass_medianas <- AgePclass_medianas %>% rename(Age=idade) %>% select(Pclass, Sex, Age)
trainMissingAge <- filter(titanic_teste, is.na(Age)) %>% select(PassengerId, Pclass, Sex, Age)
PassAge <- left_join(trainMissingAge, AgePclass_medianas, by = c("Sex","Pclass")) %>% select(PassengerId, Age=Age.y)
train.rawAge <- left_join(titanic_teste, PassAge, by = c("PassengerId"))
titanic_teste <- mutate(train.rawAge, Age = if_else(is.na(Age.x), Age.y, Age.x)) %>% select(-Age.x, -Age.y)
head(titanic_teste)

`summarise()` has grouped output by 'Sex'. You can override using the `.groups` argument.



,PassengerId,Survived,Pclass,Sex,SibSp,Parch,Fare,Embarked,Age
,<int>,<fct>,<int>,<chr>,<int>,<int>,<dbl>,<chr>,<dbl>
1,6,0,3,male,0,0,8.0500,S,30
2,10,0,3,male,1,0,6.4958,S,21
3,11,1,1,male,1,1,81.8583,S,53
4,12,0,3,male,0,0,7.9250,S,20
5,13,0,3,male,0,0,7.8958,S,26
6,24,0,2,male,2,0,31.5000,S,24


In [ ]:
sum(is.na(titanic_teste$Age))

[1] 0

### iii)            Removendo as linhas com dados faltantes e atribuindo valor para os NAs de Age.

#### Treino

In [ ]:
sum(is.na(titanic))

[1] 1

In [ ]:
titanic <- drop_na(titanic)
sum(is.na(titanic))

[1] 0

#### Teste

In [ ]:
sum(is.na(titanic_teste))

[1] 1

In [ ]:
titanic_teste <- drop_na(titanic_teste)
sum(is.na(titanic))

[1] 0

## b)      Criando um modelo no qual Survived será uma função das demais variáveis.

In [ ]:
head(titanic)

,PassengerId,Survived,Pclass,Sex,SibSp,Parch,Fare,Embarked,Age
,<int>,<fct>,<int>,<chr>,<int>,<int>,<dbl>,<chr>,<dbl>
1,2,0,3,male,0,0,7.8542,S,29
2,3,1,1,female,1,0,90.0000,Q,37
3,4,0,2,male,0,0,13.0000,S,36
4,5,1,1,female,0,1,55.0000,S,35
5,15,1,2,male,0,0,10.5000,S,62
6,17,1,3,male,0,0,7.5500,S,24


In [ ]:
modelo <- glm(formula = Survived  ~ Pclass + Sex + Age + SibSp + Parch + Fare + Embarked, family = binomial(), data = titanic)
summary(modelo)


Call:
glm(formula = Survived ~ Pclass + Sex + Age + SibSp + Parch + 
    Fare + Embarked, family = binomial(), data = titanic)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3972  -0.6915  -0.5079   0.7080   2.3005  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)  4.053611   0.630768   6.426 1.31e-10 ***
Pclass      -0.747076   0.163423  -4.571 4.84e-06 ***
Sexmale     -2.546412   0.235950 -10.792  < 2e-16 ***
Age         -0.027049   0.008722  -3.101  0.00193 ** 
SibSp       -0.342674   0.123568  -2.773  0.00555 ** 
Parch       -0.081500   0.123596  -0.659  0.50963    
Fare         0.003431   0.002498   1.374  0.16950    
EmbarkedQ   -0.529824   0.430329  -1.231  0.21825    
EmbarkedS   -0.451552   0.263068  -1.716  0.08607 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 832.39  on 622  degrees of freedom
Residual deviance: 601.

## c)       Avaliando o modelo, verificando a significância estatística das variáveis

### Avaliando os coeficientes do modelo

Estes são os coeficientes do modelo. Podemos avaliar a partir deles que um aumento de 1 ano na idade sugere uma redução por um fator de 0,97 na chance de sobrevivência.

também que ser do sexo masculino também reduz em 2,64% a chance de vida, assim como estar na Pclass 2 ou 3 sugere que há uma chance menor de sobrevivência do que estar na 1

In [ ]:
  coeficientes <- data.frame(log = coef(modelo))
  coeficientes['odds'] <- round(exp(coeficientes$log),3)
  coeficientes['probab'] <- round(exp(coeficientes$log)/(1 + coeficientes$log),3)
  coeficientes

,log,odds,probab
,<dbl>,<dbl>,<dbl>
(Intercept),4.05361066,57.605,11.399
Pclass,-0.74707567,0.474,1.873
Sexmale,-2.54641158,0.078,-0.051
Age,-0.02704865,0.973,1.000
SibSp,-0.34267368,0.710,1.080
Parch,-0.08150012,0.922,1.004
Fare,0.00343121,1.003,1.000
EmbarkedQ,-0.52982386,0.589,1.252
EmbarkedS,-0.45155159,0.637,1.161


In [ ]:
deviance(modelo) / df.residual(modelo)

[1] 0.9791779

## d)      Atualizando o modelo como consequência da análise anterior

### Modelo atualizado com menos variáveis

Verificamos que o modelo possui variáveis sem significância estatística, então as removemos em uma nova versão do modelo:

In [ ]:
modelo_atualizado <- glm(formula = Survived  ~Pclass + Sex + Age + SibSp + Embarked, family = binomial(), data = titanic)
summary(modelo_atualizado)


Call:
glm(formula = Survived ~ Pclass + Sex + Age + SibSp + Embarked, 
    family = binomial(), data = titanic)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-2.3334  -0.6981  -0.4976   0.7085   2.3100  

Coefficients:
             Estimate Std. Error z value Pr(>|z|)    
(Intercept)  4.437708   0.555689   7.986 1.39e-15 ***
Pclass      -0.860856   0.142210  -6.053 1.42e-09 ***
Sexmale     -2.530979   0.228018 -11.100  < 2e-16 ***
Age         -0.027099   0.008676  -3.123  0.00179 ** 
SibSp       -0.330394   0.117446  -2.813  0.00491 ** 
EmbarkedQ   -0.555034   0.424215  -1.308  0.19074    
EmbarkedS   -0.537813   0.255094  -2.108  0.03501 *  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

(Dispersion parameter for binomial family taken to be 1)

    Null deviance: 832.39  on 622  degrees of freedom
Residual deviance: 603.43  on 616  degrees of freedom
AIC: 617.43

Number of Fisher Scoring iterations: 4


### Avaliando a anova para comparar as versões do modelo:

In [ ]:
anova(modelo_atualizado, modelo, test = "Chisq")

,Resid. Df,Resid. Dev,Df,Deviance,Pr(>Chi)
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,616,603.4302,NA,NA,NA
2,614,601.2152,2,2.214922,0.3303967


O valor de p acima de 0,05 (p = 0,33) sujere que o modelo reduzido se adequa tão bem quanto o modelo completo com todas as variáveis, neste caso adotamos o modelo mais simples.

### Avaliando a superdispersão

A Super Dispersão (Overdispersion) ocorre quando a variância observada da variável resposta é maior do que seria esperado de uma distribuição binomial.

In [ ]:
deviance(modelo_atualizado) / df.residual(modelo_atualizado)

[1] 0.9795944

O valor de 0,98 está muito próximo de 1 e isso indica que não há superdispersão

### Fazedo previsões

In [ ]:
df <- titanic_teste

In [ ]:
previsao_pct <- predict(modelo_atualizado, type = "response", newdata = titanic_teste)
df['previsao_pct'] <- round(previsao_pct,3)

In [ ]:
previsao <- if_else(previsao_pct >= 0.5, 1, 0)
previsao <- factor(previsao, levels = c("0", "1"), labels = c("morto","vivo"))

In [ ]:
df['estimado'] <- previsao
df['Survived'] <- factor(df$Survived, levels = c("0", "1"), labels = c("morto","vivo"))

In [ ]:
head(select(df, Survived, previsao_pct, estimado),10)

,Survived,previsao_pct,estimado
,<fct>,<dbl>,<fct>
1,morto,0.116,morto
2,morto,0.108,morto
3,vivo,0.221,morto
4,morto,0.147,morto
5,morto,0.128,morto
6,morto,0.159,morto
7,vivo,0.905,vivo
8,vivo,0.881,vivo
9,vivo,0.926,vivo


### Matriz de confusão

Acurácia, Sensibilidade e Especificidade

Em modelos de classificação ou modelos de regressão logística (binários), temos dois termos muito utilizados para caracterizar o desempenho do modelo: sensibilidade e especificidade.

Sensibilidade é sinônimo de precisão.

Contudo, a especificidade é o percentual de negativos corretamente identificados.

In [ ]:
matconf <- table( df$Survived, previsao, dnn = list("Previsto", "Real"))
matconf

        Real
Previsto morto vivo
   morto   143   30
   vivo     22   71

In [ ]:
VP <- matconf[4]
VN <- matconf[1]
FP <- matconf[2]
FN <- matconf[3]
P <- sum(df$estimado == 'vivo')
N <- sum(df$estimado == 'morto')

In [ ]:
glue('Verdadeiro Positivo = {VP}')
glue('Verdadeiro Negativo = {VN}')
glue('Falso Positivo =      {FP}')
glue('Falso Negativo =      {FN}')
glue('Número de eventos = {P}')
glue('Número de não eventos = {N}')

Verdadeiro Positivo = 71

Verdadeiro Negativo = 143

Falso Positivo =      22

Falso Negativo =      30

Número de eventos = 101

Número de não eventos = 165

Acurácia = (VN + VP) / (P + N)
É a proporção de predições corretas. 

In [ ]:
acuracia <- (VN + VP) / (P + N) 
round(acuracia,3)

[1] 0.805

Ao compararmos os resultados de estimados pelo modelo com os resutados reais da base de treino temos que houve acerto em 80% dos casos.

Sensibilidade = VP / (VP + FN)
É a proporção de verdadeiros positivos. 

In [ ]:
sensibilidade <- VP / (VP + FN)
round(sensibilidade,3)

[1] 0.703

Na previsão de sobrevivência, modelo criado acertou 70% dos casos em que houve sobrevivência 

Especificidade = VN / (VN + FP)
É a proporção de verdadeiros negativos.

In [ ]:
especificidade <- VN / (VN + FP)
round(especificidade, 3)

[1] 0.867

Ao avaliarmos os casos de mortes o modelo foi ainda mais assertivo e previu corretamente 87% dos casos

## e)      Fazendo as previsões da variável Survived na base de dados não rotulada utilizando o modelo refinado, e preparando um arquivo CSV para submissão.

In [ ]:
df_teste <- read.csv('/content/titanic_dw_test.csv')
head(df_teste,3)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
,<int>,<int>,<chr>,<chr>,<dbl>,<int>,<int>,<chr>,<dbl>,<chr>,<chr>
1,1,2,"Stanton, Mr. Samuel Ward",male,41,0,0,237734,15.0458,NA,C
2,7,3,"Lang, Mr. Fang",male,26,0,0,1601,56.4958,NA,S
3,8,1,"Eustis, Miss. Elizabeth Mussey",female,54,1,0,36947,78.2667,D20,C


In [ ]:
summary(df_teste)

  PassengerId         Pclass          Name               Sex           
 Min.   :   1.0   Min.   :1.000   Length:418         Length:418        
 1st Qu.: 313.2   1st Qu.:2.000   Class :character   Class :character  
 Median : 638.5   Median :3.000   Mode  :character   Mode  :character  
 Mean   : 652.7   Mean   :2.299                                        
 3rd Qu.: 983.8   3rd Qu.:3.000                                        
 Max.   :1309.0   Max.   :3.000                                        
                                                                       
      Age            SibSp            Parch           Ticket         
 Min.   : 0.75   Min.   :0.0000   Min.   :0.0000   Length:418        
 1st Qu.:21.00   1st Qu.:0.0000   1st Qu.:0.0000   Class :character  
 Median :28.00   Median :0.0000   Median :0.0000   Mode  :character  
 Mean   :29.63   Mean   :0.4354   Mean   :0.2871                     
 3rd Qu.:38.00   3rd Qu.:1.0000   3rd Qu.:0.0000                     
 Max

In [ ]:
# Survived, Pclass, Sex, Age, SibSp, Parch, Fare, Embarked
df_teste['Name'] <- NULL
df_teste['Ticket'] <- NULL
df_teste['Cabin'] <- NULL

In [ ]:
AgePclass_medianas <- df_teste %>% group_by(Sex,Pclass) %>% summarise(idade = median(Age, na.rm = T),) %>% arrange(Pclass) %>% ungroup()
AgePclass_medianas <- AgePclass_medianas %>% rename(Age=idade) %>% select(Pclass, Sex, Age)
trainMissingAge <- filter(df_teste, is.na(Age)) %>% select(PassengerId, Pclass, Sex, Age)
PassAge <- left_join(trainMissingAge, AgePclass_medianas, by = c("Sex","Pclass")) %>% select(PassengerId, Age=Age.y)
train.rawAge <- left_join(df_teste, PassAge, by = c("PassengerId"))
df_teste <- mutate(train.rawAge, Age = if_else(is.na(Age.x), Age.y, Age.x)) %>% select(-Age.x, -Age.y)
head(df_teste$Age)

`summarise()` has grouped output by 'Sex'. You can override using the `.groups` argument.



[1] 41 26 54 48 62 36

In [ ]:
sum(is.na((df_teste$Age)))

[1] 0

In [ ]:
df_teste <- drop_na(df_teste)
sum(is.na(df_teste))

[1] 0

In [ ]:
previsao_teste <- predict(modelo_atualizado, type = "response", newdata = df_teste)
previsao <- if_else(previsao_teste >= 0.5, 1, 0)
previsao <- factor(previsao, levels = c("0", "1"), labels = c("0","1"))

In [ ]:
df_teste['previsao_pct'] <- round(previsao_teste,2)
df_teste['Survived'] <- previsao

In [ ]:
head(df_teste)

,PassengerId,Pclass,Sex,SibSp,Parch,Fare,Embarked,Age,previsao_pct,Survived
,<int>,<int>,<chr>,<int>,<int>,<dbl>,<chr>,<dbl>,<dbl>,<fct>
1,1,2,male,0,0,15.0458,C,41,0.28,0
2,7,3,male,0,0,56.4958,S,26,0.13,0
3,8,1,female,1,0,78.2667,C,54,0.86,1
4,9,1,female,1,1,79.2000,C,48,0.87,1
5,14,1,male,0,0,26.5500,S,62,0.24,0
6,16,1,female,0,2,71.0000,S,36,0.89,1


### Entrega

In [ ]:
entrega <- select(df_teste, 'PassengerId', 'Survived')
head(entrega)

,PassengerId,Survived
,<int>,<fct>
1,1,0
2,7,0
3,8,1
4,9,1
5,14,0
6,16,1


In [ ]:
str(entrega)

'data.frame':	417 obs. of  2 variables:
 $ PassengerId: int  1 7 8 9 14 16 20 23 29 37 ...
 $ Survived   : Factor w/ 2 levels "0","1": 1 1 2 2 1 2 1 1 1 1 ...


In [ ]:
summary(entrega)

  PassengerId     Survived
 Min.   :   1.0   0:261   
 1st Qu.: 313.0   1:156   
 Median : 636.0           
 Mean   : 652.2           
 3rd Qu.: 984.0           
 Max.   :1309.0           

In [ ]:
sum(is.na(entrega))

[1] 0

## f)        Criando um arquivo CSV para submissão da classificação. (apenas com o índice e o resultado da classificação)

In [ ]:
write.table(entrega, file = "entrega.csv", sep = ",", quote = TRUE, row.names = FALSE, eol = "\r\n")